<a href="https://colab.research.google.com/github/hyattjon/econ484/blob/master/examples/movies%20IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
!git clone https://github.com/brighamfrandsen/econ484.git
%cd econ484/utilities
from preamble import *
%cd ../data


Cloning into 'econ484'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 445 (delta 110), reused 37 (delta 37), pack-reused 288 (from 1)
Receiving objects: 100% (445/445), 14.02 MiB | 6.66 MiB/s, done.
Resolving deltas: 100% (199/199), done.
/content/econ484/utilities
/content/econ484/data


In [12]:
!pip install weak_instruments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.1 MB/s eta 0:00:00


# Example: Instrumental Variables Estimation of the Effect of Social Spillovers on Movie-going

This notebook will illustrate the entire supervised machine learning process in the context of predicting movie attendance based on the weather on opening weekend.

### Figure out your question

What is the effect of opening-weekend attendance on subsequent weekend attendance at a movie?

## Obtain a labeled dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
moviedata=pd.read_csv('./movies_cleaned.csv')
print(moviedata.head())
print("Shape: {}".format(str(moviedata.shape)))

   x_openingsales  open_res_own_snow_6  open_res_own_snow_0  \
0            0.44             3.55e-02             2.87e-02   
1            2.06             1.41e-03            -7.78e-04   
2            1.52             3.93e-03             1.13e-02   
3            5.89             1.05e-01             1.26e-01   
4            3.57             7.20e-02             9.29e-02   

   open_res_own_rain_6  ...  open_res_own_prec_4_0  open_res_own_prec_5_6  \
0                 0.11  ...               2.34e-02               1.36e-02   
1                 0.04  ...              -1.08e-02              -2.52e-02   
2                 0.12  ...              -1.23e-02              -1.97e-03   
3                 0.15  ...               8.02e-03               6.22e-02   
4                -0.05  ...              -2.46e-03              -4.70e-03   

   open_res_own_prec_5_0  y_ticketsales  
0               6.82e-02           0.23  
1              -2.41e-02           1.29  
2              -1.67e-02        

Let's define our "label" (y) vector, our "treatment" vector (d), and our instrument matrix (Z):

In [4]:
y = moviedata.loc[:,'y_ticketsales']
d = moviedata.loc[:,['x_openingsales']]
Z = moviedata.filter(like='open_',axis=1)
print('our y vector is:\n',y.head)
print('our d vector is:\n',d.head)
print('our instrument matrix is:\n',Z.head)

our y vector is:
 <bound method NDFrame.head of 0       0.23
1       1.29
2       0.97
3       3.22
4       2.35
        ... 
1666    3.46
1667    3.37
1668    1.83
1669    1.15
1670    1.05
Name: y_ticketsales, Length: 1671, dtype: float64>
our d vector is:
 <bound method NDFrame.head of       x_openingsales
0               0.44
1               2.06
2               1.52
3               5.89
4               3.57
...              ...
1666            9.75
1667            6.29
1668            2.91
1669            1.67
1670            3.03

[1671 rows x 1 columns]>
our instrument matrix is:
 <bound method NDFrame.head of       open_res_own_snow_6  open_res_own_snow_0  open_res_own_rain_6  \
0                3.55e-02             2.87e-02                 0.11   
1                1.41e-03            -7.78e-04                 0.04   
2                3.93e-03             1.13e-02                 0.12   
3                1.05e-01             1.26e-01                 0.15   
4                7.2

## Start with OLS of y on d. Be sure to print import necessary packages and print out coefficients!

### Try yourself first!

### Cheat if you need to

In [5]:
from sklearn import linear_model

ols = linear_model.LinearRegression().fit(d,y)
print('OLS coefficient: ',ols.coef_)

OLS coefficient:  [0.471]


## Now do "manual" two-stage least squares where you first regress d on Z, obtain predicted values, then regress y on the predicted values. Be sure to print out final coefficient on d-hat!

### Try yourself first

### Cheat if you need to

In [6]:

ols_fs = ols.fit(Z,d)
dhat = ols_fs.predict(Z)
tsls = ols.fit(dhat,y)

print('2SLS coefficient: ',tsls.coef_)


2SLS coefficient:  [0.552]


## Now do ML-augmented two-stage least squares using Random Forest to obtain the fitted values

###Try yourself first

In [7]:
# import necessary packages and create prediction "object"

# first grow random forest:

# now get random forest predictions to use as instrument:

# do "first stage" using random forest predictions as instrument:

# finally, 2nd stage regression:


### Cheat

In [8]:
# import necessary packages and create prediction "object"
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=2,max_features='sqrt')
# first grow random forest:
rf.fit(Z,d)
# now get random forest predictions to use as instrument:
dtilde=rf.predict(Z).reshape(-1,1)
# do "first stage" using random forest predictions as instrument:
fs_rf=ols.fit(dtilde,d)
dhat_rf=fs_rf.predict(dtilde)
# finally, 2nd stage regression:
tsls_rf=ols.fit(dhat_rf,y)
print('2SLS+Random Forest coefficient: ',tsls_rf.coef_)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


2SLS+Random Forest coefficient:  [0.456]


##Now do Belloni, Chernozhukov, Hansen Post-Lasso 2SLS

### Try yourself first

In [9]:
# hint 1: don't forget to scale the Zs before doing Lasso
# hint 2: to select the columns of a matrix corresponding to a set of nonzero coefficients, you can do something like:
# Z_selected = Z[:,model.coef_!=0]

### Cheat

In [10]:
# Lasso tends to work better with standardized variables, so:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Z)
Z_scaled = scaler.transform(Z)

# create lasso object, setting the penalty parameter
lasso=linear_model.LassoCV()

# predict d using Z_scaled:
lasso.fit(Z_scaled,d)
print(lasso.coef_)
# grab just the Zs with nonzero coeffs
Z_selected=Z_scaled[:,lasso.coef_!=0]

# do the first stage regression via OLS using the selected Zs and get the fitted values:
postlasso_fs = ols.fit(Z_selected,d)
dhat_postlasso = postlasso_fs.predict(Z_selected)

# do 2nd stage regression using the post-lasso fitted values:
tsls_postlasso = ols.fit(dhat_postlasso,y)
print('Post-Lasso 2SLS coefficient: ',tsls_postlasso.coef_)
print(lasso.coef_)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:1664: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[-0. -0. -0. -0.  0. -0.  0. -0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  0.
  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0. -0. -0. -0.]
Post-Lasso 2SLS coefficient:  [0.505]
[-0. -0. -0. -0.  0. -0.  0. -0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0.  0.  0.
  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0. -0. -0. -0.]


In [11]:
Z_selected.shape

(1671, 1)

## Now use JIVE from Angrist, Imbens, and Kreuger

### Cheat

In [17]:
# Import the package
from weak_instruments import ujive1

#
result = ujive1.UJIVE1(Y=y.values,X=d.values,Z=Z.values)
result.summary()


UJIVE1 Regression Results
 Coefficient  Std. Error  t-stat  P>|t|  Conf. Int. Low  Conf. Int. High
       -0.10        0.12   -0.88   0.38           -0.33             0.13
        0.57        0.05   11.78   0.00            0.48             0.67
--------------------------------------------------------------------------------
R-squared: 0.705399
Adjusted R-squared: 0.705222
F-statistic: 6198.408224
Root MSE: 0.460463


## Now go back to ML-augmented 2SLS and try with several different prediction methods